In [1]:
LIGHT_IMPORT = True

%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [7]:
df.columns.tolist()

['user_id',
 'age',
 'status',
 'sex',
 'orientation',
 'body_type',
 'diet',
 'drinks',
 'drugs',
 'education',
 'ethnicity',
 'height',
 'income',
 'job',
 'last_online',
 'location',
 'offspring',
 'pets',
 'religion',
 'sign',
 'smokes',
 'speaks',
 'essay0',
 'essay1',
 'essay2',
 'essay3',
 'essay4',
 'essay5',
 'essay6',
 'essay7',
 'essay8',
 'essay9',
 'bio_text',
 'lat',
 'lon',
 'diet_c',
 'drinks_c',
 'smokes_c',
 'drugs_c']

In [3]:
df.head(5)

,user_id,age,status,sex,orientation,body_type,diet,drinks,drugs,education,...,essay7,essay8,essay9,bio_text,lat,lon,diet_c,drinks_c,smokes_c,drugs_c
0,0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,...,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,37.653540,-122.416866,omnivore,light,light,none
1,1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,...,,i am very open and will share just about anyth...,,i am a chef: this is what that means. 1. i am ...,37.804456,-122.271356,other,heavy,none,sometimes
2,2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,...,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...","i'm not ashamed of much, but writing public te...",37.779259,-122.419329,omnivore,light,none,
3,3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,...,,,you feel so inclined.,i work in a library and go to school. . . <e> ...,37.870839,-122.272863,vegetarian,light,none,
4,4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,...,,,,hey how's it going? currently vague on the pro...,37.779259,-122.419329,,light,none,none


In [14]:
import pandas as pd

# Take only the first 10 characters (YYYY-MM-DD)
df["last_online_clean"] = df["last_online"].astype(str).str.slice(0, 10)

# Parse as date
df["last_online_dt"] = pd.to_datetime(df["last_online_clean"], errors="coerce", utc=True)

# Now build recency features
df["recency_rank"] = df["last_online_dt"].rank(ascending=True)        # oldest=1
mask = df["last_online_dt"].notna()

# Normalized 0–1 scale within dataset
min_dt, max_dt = df["last_online_dt"].min(), df["last_online_dt"].max()
if pd.notna(min_dt) and pd.notna(max_dt) and max_dt > min_dt:
    span = (max_dt - min_dt).total_seconds()
    df["recency_norm"] = ((df["last_online_dt"] - min_dt).dt.total_seconds() / span).fillna(0.0)
else:
    df["recency_norm"] = 0.0

print(df[["last_online_dt","recency_norm"]].head(10))

             last_online_dt  recency_norm
0 2012-06-28 00:00:00+00:00      0.991892
1 2012-06-29 00:00:00+00:00      0.994595
2 2012-06-27 00:00:00+00:00      0.989189
3 2012-06-28 00:00:00+00:00      0.991892
4 2012-06-27 00:00:00+00:00      0.989189
5 2012-06-29 00:00:00+00:00      0.994595
6 2012-06-25 00:00:00+00:00      0.983784
7 2012-06-29 00:00:00+00:00      0.994595
8 2012-06-29 00:00:00+00:00      0.994595
9 2012-06-28 00:00:00+00:00      0.991892


In [25]:
def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [85]:
def tinder_score(user_id, k=20, pool_k=500, max_km=50,drinks_strict=False, smokes_strict=False, 
                 drugs_strict=False,allow_missing=True,weights=None):
    """
    Tinder-style scorer:
      - Hard filters: orientation, location (<= max_km), age
      - Soft features: text_sim (from semantic score), lifestyle_score, age_score, loc_score
      - Weighted blend -> final_score
    """
    if weights is None:
        weights = {"text":0.70, "age":0.10, "life":0.15, "loc":0.05}

    # 1) Candidate pool (no hard lifestyle gates)
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True,use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=True).copy()

    # 2) Component scores
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=allow_missing)
    cands = _ensure_text_sim(cands)  # maps 'score' -> 'text_sim' if present

    # 3) Blend
    ranked = apply_weights(cands, w_text=weights["text"], w_age=weights["age"],w_life=weights["life"], 
                           w_loc=weights["loc"]).copy()

    # 4) Final shape
    want = ["user_id","final_score","text_sim","age_score","lifestyle_score","loc_score","distance_km",
            "location","age","sex","orientation"]
    for col in want:
        if col not in ranked.columns:
            ranked[col] = np.nan
    ranked_top = ranked[want].head(k).copy()
    ranked_top["source"] = "ranked"

    # align random suggestions to same columns
    for col in want:
        if col not in random_potential_matches.columns:
            random_potential_matches[col] = np.nan
    return ranked[want].head(k).reset_index(drop=True)

In [86]:
tinder = tinder_score(user_id=14, k=10, pool_k=500, max_km=50)
tinder.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,final_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,38159,0.819912,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight
1,40022,0.815415,0.736307,1.0,1.0,1.0,23.920071,"san francisco, california",25,m,straight
2,3717,0.813915,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight
3,28563,0.813037,0.732910,1.0,1.0,1.0,23.920071,"san francisco, california",29,m,straight
4,39450,0.812803,0.732575,1.0,1.0,1.0,23.920071,"san francisco, california",25,m,straight
5,23663,0.811652,0.730931,1.0,1.0,1.0,29.764458,"san mateo, california",27,m,straight
6,15046,0.810298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight
7,33656,0.810048,0.728639,1.0,1.0,1.0,23.920071,"san francisco, california",26,m,straight
8,16398,0.808185,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight
9,35672,0.807806,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight


In [80]:
# Merge recency_norm, blend, and nudge active users

def apply_weights_tinder(cands,w_text=0.60, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.10):
    """
    final_score = w_text*text_sim + w_age*age_score + w_life*lifestyle_score + w_loc*loc_score + w_rec*recency_norm
    (weights are normalized inside)
    """
    c = cands.copy()
    # ensure required cols exist
    for col in ["text_sim","age_score","lifestyle_score","loc_score"]:
        if col not in c.columns:
            c[col] = 0.0

    # bring in recency_norm from the master df
    c = c.merge(df[["user_id","recency_norm"]], on="user_id", how="left")
    c["recency_norm"] = c["recency_norm"].fillna(0.0).clip(0,1)

    # normalize weights
    total = float(w_text + w_age + w_life + w_loc + w_rec)
    w_text, w_age, w_life, w_loc, w_rec = [w/total for w in (w_text, w_age, w_life, w_loc, w_rec)]

    c["tinder_score"] = (w_text*c["text_sim"] + w_age *c["age_score"] + w_life*c["lifestyle_score"] +
                        w_loc *c["loc_score"] + w_rec *c["recency_norm"])
    return c.sort_values("tinder_score", ascending=False)


In [79]:
scored = compute_component_scores(user_id=u, cands=cands, max_km=20,
    drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)
scored = _ensure_text_sim(scored)
scored.head(15)

,user_id,age,sex,orientation,location,text_sim,diet_c,drinks_c,smokes_c,drugs_c,lat,lon,age_score,lifestyle_score,loc_score,distance_km,dist_penalty
0,58565,24,f,straight,"alameda, california",0.664278,omnivore,light,none,none,37.609029,-121.899142,1.0,1.000000,0.000000,45.859473,25.859473
8,43841,24,f,straight,"san leandro, california",0.654167,omnivore,light,none,none,37.724930,-122.156077,1.0,1.000000,0.785908,24.281842,4.281842
22,47209,28,f,straight,"san francisco, california",0.644579,,heavy,none,none,37.779259,-122.419329,1.0,1.000000,1.000000,13.980956,0.000000
27,57311,28,f,straight,"san francisco, california",0.642219,,heavy,none,sometimes,37.779259,-122.419329,1.0,0.666667,1.000000,13.980956,0.000000
30,49936,28,f,straight,"oakland, california",0.641363,vegan,light,none,none,37.804456,-122.271356,1.0,0.750000,0.944814,21.103721,1.103721
31,45885,22,f,straight,"walnut creek, california",0.641020,omnivore,light,none,none,37.902073,-122.061870,1.0,1.000000,0.000000,41.679140,21.679140
33,27797,25,f,bisexual,"san francisco, california",0.640599,,,,none,37.779259,-122.419329,1.0,1.000000,1.000000,13.980956,0.000000
37,15647,24,f,straight,"san francisco, california",0.639417,omnivore,light,light,none,37.779259,-122.419329,1.0,1.000000,1.000000,13.980956,0.000000
41,44908,23,f,bisexual,"san francisco, california",0.636920,,light,heavy,none,37.779259,-122.419329,1.0,1.000000,1.000000,13.980956,0.000000
42,21873,25,f,straight,"pleasant hill, california",0.636885,,light,none,none,37.947979,-122.060796,1.0,1.000000,0.000000,45.283826,25.283826


In [73]:
ranked = apply_weights_tinder(scored, w_text=0.55, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.15)
ranked[["user_id","tinder_score","text_sim","age_score","lifestyle_score","loc_score","recency_norm","distance_km"]].head(10)

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,recency_norm,distance_km
8,44908,0.799900,0.636920,1.0,1.0,1.000000,0.997297,13.980956
11,4120,0.797826,0.633149,1.0,1.0,1.000000,0.997297,13.980956
16,57214,0.794873,0.627041,1.0,1.0,1.000000,1.000000,13.980956
27,56064,0.791719,0.621308,1.0,1.0,1.000000,1.000000,13.980956
2,47209,0.790329,0.644579,1.0,1.0,1.000000,0.905405,13.980956
26,7169,0.790105,0.621321,1.0,1.0,1.000000,0.989189,18.918516
15,26536,0.789406,0.627873,1.0,1.0,0.944814,0.997297,21.103721
19,30403,0.788710,0.624682,1.0,1.0,1.000000,0.967568,13.980956
1,43841,0.787977,0.654167,1.0,1.0,0.785908,0.997297,24.281842
35,53269,0.787719,0.615508,1.0,1.0,1.000000,0.994595,13.980956


In [101]:
u       = 14   
pool_k  = 500
max_km  = 50
k       = 10
random_sample = 10

# 1) Candidate pool (only hard gates: orientation, age, location)
cands = get_matches(user_id=u, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
    use_age=True,use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
    drinks_strict=False, smokes_strict=False, drugs_strict=False,allow_missing=True).copy()

# 2) Component scores (no recency used here)
cands = compute_component_scores(user_id=u, cands=cands, max_km=max_km,
        drinks_strict=False, smokes_strict=False, drugs_strict=False,allow_missing=True)
cands = _ensure_text_sim(cands)  # maps 'score' -> 'text_sim' if present

WANT_COLS = ["user_id","text_sim","age_score","lifestyle_score","loc_score","distance_km",
             "location","age","sex","orientation"]
    
# 3) Sample a few random potentials from this *scored* pool (before final ranking)
random_potential_matches = cands.sample(n=min(random_sample, len(cands)), random_state=42)[WANT_COLS].copy()
random_potential_matches["source"] = "suggested"

print("Pool size:", len(cands), "| sampled suggestions:", len(random_potential_matches))

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225
Pool size: 500 | sampled suggestions: 10


In [102]:
# 4) Rank with recency (linear blend)
ranked = apply_weights_tinder(cands,w_text=0.65, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.05).copy()
random_potential_matches = apply_weights_tinder(random_potential_matches,w_text=0.65, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.05).copy()

# 5) Prepare top-k ranked and merge with random suggestions
ranked_top = ranked.head(k).copy()
ranked_top["source"] = "ranked"

want = ["user_id","tinder_score","text_sim","age_score","lifestyle_score","loc_score",
        "distance_km","location","age","sex","orientation","source"]

# align columns for concat
for col in want:
    if col not in ranked_top.columns:
        ranked_top[col] = np.nan
    if col not in random_potential_matches.columns:
        random_potential_matches[col] = np.nan

tinder_suggestions = (pd.concat([ranked_top[want], random_potential_matches[want]], ignore_index=True).drop_duplicates("user_id", keep="first")
      .reset_index(drop=True))

tinder_suggestions.head(15)

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,38159,0.832641,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight,ranked
1,3717,0.826936,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight,ranked
2,15046,0.823713,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight,ranked
3,16398,0.821751,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight,ranked
4,35672,0.821399,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight,ranked
5,10444,0.819723,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight,ranked
6,31015,0.819674,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight,ranked
7,47397,0.819527,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight,ranked
8,41489,0.818305,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight,ranked
9,17586,0.816838,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight,ranked
